In [1]:
import pandas as pd
import numpy as np
import datetime


In [2]:
dane = pd.read_csv('Data/df_enriched.csv')
dane.head()

,Unnamed: 0,data_zdarzenia,godzina,GPS_x,GPS_y,miejscowosc,ulica,numer_domu,numer_drogi,pikietaz,...,zachowanie_kierujacego_niestos_do_sygn,zachowanie_kierujacego_nieust_przejazdu,zachowanie_kierujacego_nieust_pieszemu,zachowanie_kierujacego_gwalt_hamowanie,zachowanie_kierujacego_niepr_wyprzedzanie,zachowanie_kierujacego_niepr_zatrzymanie,zachowanie_kierujacego_niepr_zawracanie,zachowanie_kierujacego_niepr_przejazd_rowerowy,zachowanie_kierujacego_jazda_bez_swiatel,zachowanie_kierujacego_inne_przyczyny
0,0,2016-01-01,5,18.635806,50.369722,GLIWICE,TOSZECKA,NaN,901,61.6,...,0,0,0,0,0,0,0,0,0,0
1,1,2016-01-01,0,19.180833,50.323083,DĄBROWA GÓRNICZA,SIENKIEWICZA,NaN,G200 030S,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,2016-01-01,18,18.877639,50.445083,TARNOWSKIE GÓRY,KS.IGNACEGO SIWCA,4A,GMINNA,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,2016-01-01,15,18.860583,50.444667,TARNOWSKIE GÓRY,JÓZEFA PIŁSUDSKIEGO,6,P3276S,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,2016-01-01,17,19.047556,50.466806,OŻAROWICE,DWORCOWA,16,P3203S,NaN,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#przekszłcam nazwy miejscowosci aby ułatwić wysyłanie zapytań na stronę www.worldweatheronline.com


def simplify(a):  
    a = a.lower()
    a = a.replace(' ','_')
    a = a.replace('ą','a')                   
    a = a.replace('ę','e')                  
    a = a.replace('ć','c')                    
    a = a.replace('ł','l')                    
    a = a.replace('ń','n')                   
    a = a.replace('ó','o')                  
    a = a.replace('ś','s')
    a = a.replace('ź','z')
    a = a.replace('ż','z')
          
    return(a)

def simplify_column(serie):
    a = serie.unique()                       
    a = np.array2string(a)                  
    return(simplify(a))

def simplify_cell(string):
    return(simplify(str(string)))

In [4]:
dane['miejscowosc']=dane['miejscowosc'].apply(lambda x: simplify_cell(x))

In [5]:
location_list = dane['miejscowosc']

In [6]:
location_list = location_list.unique()

In [7]:
location_list 

array(['gliwice', 'dabrowa_gornicza', 'tarnowskie_gory', ..., 'lubki',
       'nowa_rybna', 'borowce'], dtype=object)

<span style=color:blue> Pobieramy API 

In [8]:
pip install wwo-hist

Note: you may need to restart the kernel to use updated packages.


In [9]:
# instaluję pakiet

from wwo_hist import retrieve_hist_data

In [10]:
import os     #tworzę folder, w którym będę przechowywać dane o pogodzie dla podanej miejscowości

os.chdir("C:/Users/HP/DataWorkshop/Data/WHEATHER") 

In [11]:
'''frequency =  24             #częstotliwość pobierania inforamcji 1-24
start_date = '01-JAN-2016'     #data rozpoczęcia pobierania danych
end_date = '31-DEC-2019'       #data zakończenia pobierania danych
api_key = 'MY API KEY'         #klucz poboeramy po zarejestrowaniu się na stronie www.worldweatheronline.com/developer
location_list = location_list  #lista miast dla których ma zostać pobrana pogoda 

hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)'''


"frequency =  24             #częstotliwość pobierania inforamcji 1-24\nstart_date = '01-JAN-2016'     #data rozpoczęcia pobierania danych\nend_date = '31-DEC-2019'       #data zakończenia pobierania danych\napi_key = 'MY API KEY'         #klucz poboeramy po zarejestrowaniu się na stronie www.worldweatheronline.com/developer\nlocation_list = location_list  #lista miast dla których ma zostać pobrana pogoda \n\nhist_weather_data = retrieve_hist_data(api_key,\n                                location_list,\n                                start_date,\n                                end_date,\n                                frequency,\n                                location_label = False,\n                                export_csv = True,\n                                store_df = True)"

In [12]:
#odczytuję ściągnięte dane pogodowe dla miasta Katowice

kato1 = pd.read_csv('C:/Users/HP/DataWorkshop/Data/WHEATHER/KATOWICE.csv')
kato1.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2016-01-01,-2,-6,0.0,8.1,2,50,No moonrise,12:08 PM,08:44 AM,...,14,6,45,0.0,1028,-2,10,163,9,katowice
1,2016-01-02,-7,-9,0.0,8.1,2,43,12:41 AM,12:31 PM,08:44 AM,...,31,1,51,0.0,1028,-7,10,98,23,katowice
2,2016-01-03,-9,-11,0.0,4.9,2,36,01:43 AM,12:56 PM,08:43 AM,...,26,39,46,0.0,1022,-9,10,101,19,katowice
3,2016-01-04,0,-9,1.0,4.9,1,28,02:44 AM,01:22 PM,08:43 AM,...,29,64,87,1.5,1015,0,8,208,17,katowice
4,2016-01-05,-3,-8,0.6,3.0,1,21,03:46 AM,01:52 PM,08:43 AM,...,15,87,82,0.7,1005,-3,9,157,9,katowice


In [13]:
#zmieniam typ kolumny z datą

kato1['date_time'] = pd.to_datetime(kato1['date_time'])

kato1.columns

Index(['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'moon_illumination', 'moonrise', 'moonset', 'sunrise',
       'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'visibility', 'winddirDegree', 'windspeedKmph', 'location'],
      dtype='object')

In [14]:
#wybieram interesujące mnie columny

kato1 = kato1[['date_time', 'maxtempC', 'mintempC','tempC', 'visibility','pressure','moon_illumination','location']]
kato1

,date_time,maxtempC,mintempC,tempC,visibility,pressure,moon_illumination,location
0,2016-01-01,-2,-6,-2,10,1028,50,katowice
1,2016-01-02,-7,-9,-7,10,1028,43,katowice
2,2016-01-03,-9,-11,-9,10,1022,36,katowice
3,2016-01-04,0,-9,0,8,1015,28,katowice
4,2016-01-05,-3,-8,-3,9,1005,21,katowice
...,...,...,...,...,...,...,...,...
1456,2019-12-27,2,-5,2,2,1024,0,katowice
1457,2019-12-28,1,-2,1,4,1034,8,katowice
1458,2019-12-29,0,-2,0,7,1039,15,katowice
1459,2019-12-30,3,-3,3,10,1034,22,katowice


In [15]:
#z naszych danych wybieram tylko te dotyczące miasta Katowice

kato2 = dane[dane['miejscowosc']=='katowice']
type(kato2)

pandas.core.frame.DataFrame

In [16]:
kato2['data_zdarzenia'] = pd.to_datetime(kato2['data_zdarzenia'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
#grupuję nasz data frame wg daty i obliczam sumę wypadków przypadających na konkretną datę

kato2 = (kato2
    .groupby(['data_zdarzenia'])
    .agg({'liczba_wypadkow':'sum'})
    .sort_values(by='liczba_wypadkow', ascending=False)
    )

In [18]:
#łączę nasze dane z danymi pogodowymi dla Katowic

new = pd.merge(kato1, kato2, left_on='date_time', right_on='data_zdarzenia')

In [19]:
new.sample(5)

# maxtempC: maksymalna temperatura w ciągu dnia w stopniach Celsjusza
# mintempC: minimalna temperatura w ciągu dnia w stopniach Celsjusza
# tempC: temperatura w stopniach Celsjusza
# visibility: widoczność w kilometrach
# moon_illumination: oświetlenie księżyca w % (jaka część księżyca jest widoczna, 100% - księżyc w pełni)

,date_time,maxtempC,mintempC,tempC,visibility,pressure,moon_illumination,location,liczba_wypadkow
662,2017-10-24,9,5,9,10,1024,25,katowice,1
327,2016-11-23,12,6,12,10,1023,28,katowice,0
186,2016-07-05,24,13,24,10,1016,0,katowice,1
892,2018-06-11,26,17,26,10,1009,0,katowice,0
510,2017-05-25,18,9,18,10,1018,0,katowice,0


<span style=color:blue> Liczba wypadków a tempeartura

In [23]:
(new
 .groupby(['tempC'])
 .agg({'liczba_wypadkow':'sum'})
 .sort_values(by='liczba_wypadkow', ascending=False)
 )


,liczba_wypadkow
tempC,
13,50
24,49
18,44
6,43
19,43
22,43
23,42
8,41
20,40


In [21]:
(new[new['tempC'] > 0]
     .groupby(['tempC'])
     .agg({'liczba_wypadkow':'sum'})
     .sort_values(by='liczba_wypadkow', ascending=False))

,liczba_wypadkow
tempC,
13,50
24,49
18,44
6,43
22,43
19,43
23,42
8,41
20,40


In [43]:
#liczba wypadków w dni z temperaturą powyżej 0

print('Liczba wypadków w dni z temperaturą powyżej zera', new[new['tempC'] > 0]['liczba_wypadkow'].sum())
print('Liczba wszystkich wypadków w Katowicach', new['liczba_wypadkow'].sum())
print('Liczba wypadków w dni z temperaturą niższą bądź równą zero', new[new['tempC'] <= 0]['liczba_wypadkow'].sum())

Liczba wypadków w dni z temperaturą powyżej zera 992
Liczba wszystkich wypadków w Katowicach 1071
Liczba wypadków w dni z temperaturą niższą bądź równą zero 79


In [33]:
#dane dotyczące dni z temperaturą powyżej 0

new[new['tempC'] > 0].count()

date_time            1332
maxtempC             1332
mintempC             1332
tempC                1332
visibility           1332
pressure             1332
moon_illumination    1332
location             1332
liczba_wypadkow      1332
dtype: int64

In [45]:
#992 wypadki przypadające na 1332 dni z temperaturą powyżej zera

In [47]:
#dane dotyczące dni z temperaturą niższą bądź równą zero

new[new['tempC'] <=0].count()

date_time            129
maxtempC             129
mintempC             129
tempC                129
visibility           129
pressure             129
moon_illumination    129
location             129
liczba_wypadkow      129
dtype: int64

In [48]:
#79 wypadków przypadające na 129 dni z temperaturą niższą bądź równą zero 